# Final Assignment
## CM3015 Machine Learning and Neural Networks

### Credit Card Fraud detection using a feedforward Multilayer Perceptron Deep Learning Neural Network

- Student: cy150

---

### Overview: Credit Card Fraud

Credit card frauds is an aunauthorized use of a credit (often definiton also include debit) cards to make purchase, withdraw funds or to create transtions that the legitimate cardholder did not approve

### Problem Statement

Credit card graud causes direct and often significant financial losses where card issuers, merchants and consumers may absorb direct losses from unauthorized purchases and chargebacks. A fraud creates unecessary invesgative work and reviews, dispute handling and temporarily loss of funds and merchant/bank reputation
As a result of fraud, stricter verification and KYC is implemented by merchants and banks. These merchants and banks can also lose customers if there is excessive false declines or chargebacks. Additionally, failure to protect customers according to regulatory compliance standards may incccur penalities on card networks, banks or merchants

---

## Dataset Overview
This dataset consist of transactions made by credit card transactions made by European cardholders
This dataset zooms in on transactions that occured in 2 days where 492 frauds out of 284, 807 transactions take place
The databaset is described to be unbalanced with positive frauds accounting only for 0172% of all transactions


### Nature of the Dataset

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, the dataset provider was unable to provide the original features and more background information about the data. 


### Dataset Features
Features V1, V2, … V28 are the principal components are obtained with PCA (Principle Component Analysis), the only features which have not been transformed with PCA are 'Time' and 'Amount'. 

- Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset.


*Example*

| Time |      V1 |      V2 |     V3 |     V4 |      V5 |
| ---: | ------: | ------: | -----: | -----: | ------: |
|    0 | -1.3598 | -0.0728 | 2.5363 | 1.3782 | -0.3383 |
|    1 |  1.1919 |  0.2662 | 0.1665 | 0.4482 |  0.0600 |
|    1 | -1.3584 | -1.3402 | 1.7732 | 0.3798 | -0.5032 |


### Dataset Licensing

Dataset is licensed under the Database Contents License (DbCL)

According to the license, opendatacommons, the "Licensor grants to you (me or anyone) a worldwide, royalty-free, non-exclusive, perpetual, irrevocable copyright license to do any act that is restricted by copyright over anything within the Contents, whether in the original medium or any other. These rights explicitly include commercial use, and do not exclude any field of endeavour"

### Permission

The DbCL license explicitly allows me to utilise this dataset for this final assignment

### Dataset Author

- Machine Learning Group - ULB 

### Dataset Source

After browsing through Kaggle, i found a dataset that is complex, challanging but at the same time, provides with rich features that my model can learn from, and extract information

Link to Dataset ```https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/data```

### Justification for this dataset

#### Rationale behind why this dataset was chosen


- The dataset for Credit Card Fraud detection is based on real-life data and possess a technically and intellecturally challanging binary classification problem that fits naturally well with model constraints 
- The dataset allows for a full deep learning workflow due to its tabular numeric data with numerous input dimensions with binary labels allowing the model to learn from to create a accurate robust DLNN
- This dataset is also imbalanced, presenting me with ample opportunity to follow a proper deep leaazrning workflow and pushing me to produce more robust evaluation practices like precision, recall, PR, AUC and threshold selection
- This dataset possesses fixed length numeric feature vector, combined with its structured tabular structure matches the layer restriction very well, cleanly lining up with the Dense and Dropout layers that the model can learn from with little to no mismatch

#### Limitations of this dataset and my mitigation stragety

- The features of this dataset, namely V1 to V28 are anonymized PCA components whilst the time and amount column are not PCA transformed, disabling my ability to intepret each V feature systematically. To mitigate this limitation, i would focus on modelling them as informative numeric inputs instead

---

## Evaluation Criteria

### Objective

The primary objective for my neural network is to detect fradualent transactions whilst minimizing false positives. Frauds are rare so my evaluating criteria should zone in on minority class performance which means selecting an operating point that reflects the cost of errors

### Holdout Protocol

1. The data will be split into:

    - Training set
    - Validation set
    - test set

2. Preprocessing decisions are fitted using training set exclusively
3. Validation sets are only used for Model and threshold selection
4. Final performance is evaluated based on the untouched test set


## Methodology: Evaluation Metrics

This section discuss how i would measure the performance of my model based on a pre-defined set of evaluation criteria

### Primary Metrics

##### Precesion Recall AUC
- The main summary metric, it provides more information than accuracy under extreme class imbalances


##### Recall
- Measures true positives only, mirroring how fraud risks are missed

##### F1 Score
- Used as a single score, bgalancing between precision and recall, usefull for when a neutral trade off between precision and recall is required

### Secondary Metrics

1. Confusion Matrix at predetermined threshold
    - using TP, FP, TN, FN, making clear on the tradeoffs between each aspect

2. FN (False Negative) rates and FP (False Negative rates)

3. Callibration Check
    - Compare predicted probabilities with observed outcomes using simple binning table, to determine if the model outputs can be considered for meaningful probabilities

#### Justification for choice of the two evaluation metrics

The two categories of evaluation metrics mentioned above was due to the nature of this problem involves competiting objects that is hard or even impossible to be fully expressed as a singular numeric value
This is due to the nature of fraud being a rare event. This might result in a model to appear seemingly strong on one evaluation metric while still nonetheless fails at another metric in practice. The utilization of the primary metric provides me with a consistent rule for comparing model differences when testing the multiple deep learning instances while the secondary metric provides the necessary context to intepret the trade offs between false positives and false negatives

### Implications of my evaluation metrics

1. Precision reflects workload and customer frictions
2. Recall reflects loss prevention in real life, often translating into direct financial losses
3. PR AUC reflects ranking qualities under rare fraud, showing if my model rank true more often than normal transactions across multiple possible thresholds
4. The confusion matrix is chosen at a threshold where it reflects the actual transaction behavior


### Measures to ensure evaluation reflects real life performance in fraud detection

To ensure that the evaluation metrics are useful and representative of actual real life credit card fraud detection i have to ensure that:

1. I utilize a realistic split stragety. This is because some credit card transactions have time orders. Splitting up where the training uses the earlier data and where testing is done at later data better reflects deployment than a fully random split
2. Splitting first in preprocessing ensures that there is no leakage of information from the test distribution and taht my test results is reflective of unseen future data
3. I choose metrics that mirrors my real objective. This justify why i don't use Accuracy as my main success metric as it can look effective but detecting almost no fraud
4. I do not prick a threshold solely based on the test set. Clearly define an operating point using validation data, then lock the threshold and report the test performance once
5. Additionally report multiple thresholds by including a small threshold sweep table on validation set showing how precision and recall change in addition wiht the final threshold
6. Calibrate and sanity check my model output using bins
7. Monitor precision and recall over time and retrian preiodically to refect real base rates and address dift risks
8. Stress test my model with repeat trainings using different random seeds to confirm taht the results are stable

### How my evaluation approach goes beyond the syllabus

- Instead of using a standard 0.5 cutoff in threshold, i choose a decision threshold on the validation set using maximize recall subject to ensure precision to some degree
- Additionally using advance evaluation metrics such as precision recall AUC as a primary model selection metric, further allows me to go beyond the syllabus where it emphasizes more on loss and accuracy
- Moreover, running each configuration multiple times and reporting the mean and std of metrics is closer to academia research practice, far beyond typical coursework requirements
- My utilization of a time aware evaluation split shows that i understand how test performance depends on how the data is sampled, mirroring the real world
- Binning, a technique that i will utilize is also an advance technique used in deep research for technical translation and for post graduate programmes
---

## Methodology: Choosing the right model

Analyzing the schema of the dataset and understanding the problem , decided to utilize a ```feedforward multi-layer perceptron``` with a fully connected network of Sense layers and Dropout, which ultimately ends with a single sigmoid output for binary classification

#### Model Size
    - Small to medium ```feedforward multilayer perceptron```

### Overview on feedforward Neural Networks

### Overview on Multi-layer perceptrons

### Justification: Why feedforward Multilayer Perceptron

### Current Real life applications



## Glossary of terms


| Term                  | Definition                                                                                                             |
| --------------------- | ---------------------------------------------------------------------------------------------------------------------- |
| Binary classification | A prediction task with two classes, here fraud versus non fraud                                                        |
| Class imbalance       | A dataset property where one class is much rarer than the other                                                        |
| Positive class        | The class of interest, here fraud transactions labeled 1                                                               |
| Negative class        | The other class, here legitimate transactions labeled 0                                                                |
| Feature               | An input variable used for prediction, such as V1 or Amount                                                            |
| Label                 | The target variable the model learns to predict, here Class                                                            |
| PCA                   | Principal Component Analysis, a transformation that creates new variables as linear combinations of original variables |
| Principal component   | One PCA derived feature, here V1 to V28                                                                                |
| Train set             | Data used to fit model parameters                                                                                      |
| Validation set        | Data used to select hyperparameters and decision threshold                                                             |
| Test set              | Held out data used once for final performance reporting                                                                |
| Data leakage          | When information from validation or test data influences training or preprocessing decisions                           |
| Standardization       | Scaling features to have zero mean and unit variance using training statistics                                         |
| Normalization         | Rescaling features to a fixed range, often 0 to 1, depending on the method                                             |
| Model                 | A function that maps input features to a predicted output                                                              |
| Neural network        | A model composed of layers of learned transformations, here Dense and Dropout layers                                   |
| Dense layer           | A fully connected layer that applies a linear transformation followed by an activation function                        |
| Dropout               | A regularization method that randomly disables a fraction of units during training to reduce overfitting               |
| Activation function   | A non linear function applied within a layer, such as ReLU or sigmoid                                                  |
| Sigmoid               | An activation that maps a real number to a value between 0 and 1, used for binary outputs                              |
| Logits                | The raw model output before applying sigmoid                                                                           |
| Probability score     | The model output after sigmoid, interpreted as probability like score                                                  |
| Decision threshold    | The cutoff used to convert probability scores into class predictions                                                   |
| Confusion matrix      | A table counting true positives false positives true negatives and false negatives                                     |
| True positive TP      | Fraud correctly predicted as fraud                                                                                     |
| False positive FP     | Legitimate predicted as fraud                                                                                          |
| True negative TN      | Legitimate correctly predicted as legitimate                                                                           |
| False negative FN     | Fraud predicted as legitimate                                                                                          |
| Precision             | TP divided by TP plus FP, the fraction of predicted fraud that is truly fraud                                          |
| Recall                | TP divided by TP plus FN, the fraction of actual fraud that is detected                                                |
| F1 score              | Harmonic mean of precision and recall                                                                                  |
| ROC curve             | Curve of true positive rate versus false positive rate over thresholds                                                 |
| AUC                   | Area under a curve, a threshold independent performance summary                                                        |
| PR curve              | Precision versus recall over thresholds                                                                                |
| PR AUC                | Area under the precision recall curve, often preferred for rare event detection                                        |
| Overfitting           | When a model performs well on training but poorly on new data                                                          |
| Regularization        | Methods that reduce overfitting, such as dropout or weight penalties                                                   |
| Hyperparameter        | A setting chosen outside training, such as number of layers, dropout rate, learning rate                               |
| Learning rate         | Step size used by the optimizer when updating model weights                                                            |
| Optimizer             | Algorithm that updates model weights to minimize the loss, such as Adam                                                |
| Loss function         | The quantity the model minimizes during training, such as binary cross entropy                                         |
| Early stopping        | Stopping training when validation performance stops improving                                                          |
| Calibration           | How well predicted probabilities match observed event rates                                                            |
| Concept drift         | When the data generating process changes over time, causing performance degradation                                    |
| Baseline model        | A simple reference model used for comparison, such as always predicting non fraud                                      |


---
### Bilbography & Citations


Carcillo, F., Le Borgne, Y. A., Caelen, O. and Bontempi, G. (2018) ‘Streaming active learning strategies for real life credit card fraud detection: assessment and visualization’, International Journal of Data Science and Analytics, 5(4), pp. 285 300.

Carcillo, F., Dal Pozzolo, A., Le Borgne, Y. A., Caelen, O., Mazzer, Y. and Bontempi, G. (2018) ‘Scarff: a scalable framework for streaming credit card fraud detection with Spark’, Information Fusion, 41, pp. 182 194.

Carcillo, F., Le Borgne, Y. A., Caelen, O., Oblé, F. and Bontempi, G. (2019) ‘Combining unsupervised and supervised learning in credit card fraud detection’, Information Sciences.

Dal Pozzolo, A. (n.d.) Adaptive machine learning for credit card fraud detection. PhD thesis. Université libre de Bruxelles.

Dal Pozzolo, A., Caelen, O., Johnson, R. A. and Bontempi, G. (2015) ‘Calibrating probability with undersampling for unbalanced classification’, in Proceedings of the IEEE Symposium on Computational Intelligence and Data Mining. IEEE.

Dal Pozzolo, A., Caelen, O., Le Borgne, Y. A., Waterschoot, S. and Bontempi, G. (2014) ‘Learned lessons in credit card fraud detection from a practitioner perspective’, Expert Systems with Applications, 41(10), pp. 4915 4928.

Dal Pozzolo, A., Boracchi, G., Caelen, O., Alippi, C. and Bontempi, G. (2018) ‘Credit card fraud detection: a realistic modeling and a novel learning strategy’, IEEE Transactions on Neural Networks and Learning Systems, 29(8), pp. 3784 3797.

Lebichot, B., Le Borgne, Y. A., He, L., Oblé, F. and Bontempi, G. (2019) ‘Deep learning domain adaptation techniques for credit cards fraud detection’, in INNSBDDL 2019 Recent Advances in Big Data and Deep Learning, pp. 78 88.

Lebichot, B., Paldino, G., Siblini, W., He, L., Oblé, F. and Bontempi, G. (n.d.) ‘Incremental learning strategies for credit cards fraud detection’, International Journal of Data Science and Analytics.

Le Borgne, Y. A. and Bontempi, G. (n.d.) Reproducible machine learning for credit card fraud detection: practical handbook.

If you want, paste your target year for the PhD thesis and the handbook, plus any missing page ranges, and I will update the entries so everything is fully complete and consistent.
